In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print('Environment ready.')

## 1–2) Dataset Acquisition & Loading

Update `DATA_PATH` if needed. This expects the Kaggle **Seattle Airbnb Open Data** `listings.csv` file.

In [ ]:
# Set your path to the listings file here
DATA_PATH = Path('listings.csv')

if not DATA_PATH.exists():
    raise FileNotFoundError('Could not find listings.csv. Place the Kaggle Seattle Airbnb listings file in the working directory.')

df_raw = pd.read_csv(DATA_PATH, low_memory=False, on_bad_lines="skip")
print('Loaded:', DATA_PATH)
print('Shape:', df_raw.shape)
display(df_raw.head())

## 3) Data Exploration

We report: number of records & features, data types, and missing values per column.

In [ ]:
n_rows, n_cols = df_raw.shape
print(f'Records: {n_rows} | Features: {n_cols}')
print('\nData types:')
display(df_raw.dtypes)

print('\nMissing values by column:')
missing_summary = df_raw.isna().sum().sort_values(ascending=False).to_frame('missing_count')
missing_summary['missing_pct'] = missing_summary['missing_count'] / len(df_raw) * 100
display(missing_summary)

## 4) Data Visualization

We visualize:
- **Price distribution** (histogram)
- **Room type frequency** (bar chart)
- **Price vs. number of reviews** (scatter plot)

> If your dataset uses different column names, adjust the variables below accordingly.

In [ ]:
# Choose common columns from the Seattle dataset
PRICE_COL = 'price'
ROOM_TYPE_COL = 'room_type'
REVIEWS_COL = 'number_of_reviews'

def parse_price(s):
    if pd.isna(s):
        return np.nan
    # Handles strings like "$1,234.00"
    return float(str(s).replace('$','').replace(',','').strip())

prices = df_raw[PRICE_COL].map(parse_price)

# 1) Histogram of price
plt.figure()
plt.hist(prices.dropna(), bins=50)
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# 2) Bar chart of room types
if ROOM_TYPE_COL in df_raw.columns:
    counts = df_raw[ROOM_TYPE_COL].value_counts().sort_values(ascending=False)
    plt.figure()
    counts.plot(kind='bar')
    plt.title('Room Type Frequencies')
    plt.xlabel('Room Type')
    plt.ylabel('Count')
    plt.show()
else:
    print(f'Column {ROOM_TYPE_COL} not found; skipping bar chart.')

# 3) Scatter: price vs number of reviews
if REVIEWS_COL in df_raw.columns:
    plt.figure()
    plt.scatter(df_raw[REVIEWS_COL], prices)
    plt.title('Price vs Number of Reviews')
    plt.xlabel('Number of Reviews')
    plt.ylabel('Price')
    plt.show()
else:
    print(f'Column {REVIEWS_COL} not found; skipping scatter plot.')

## 5) Data Preprocessing

Steps:
- Parse and clean `price` (target).
- Select a useful subset of features (numerical + categorical).
- Impute missing values.
- One-hot encode categoricals (handle unknowns).
- Scale numerical features (for SVR).
- Train/test split (stratified not typical for regression; we do random split).

In [ ]:
# Parse target
df = df_raw.copy()
df['price_clean'] = df[PRICE_COL].map(parse_price)

# Basic feature set (feel free to extend)
candidate_numeric = [c for c in [
    'accommodates','bedrooms','bathrooms','bathrooms_text','beds',
    'minimum_nights','maximum_nights','availability_30','availability_365',
    'number_of_reviews','review_scores_rating','review_scores_accuracy',
    'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
    'review_scores_location','review_scores_value','calculated_host_listings_count'
] if c in df.columns]

# bathrooms_text is not numeric; we'll parse a float if possible
def parse_bathrooms_text(val):
    if pd.isna(val):
        return np.nan
    s = str(val).split(' ')[0]
    try:
        return float(s)
    except Exception:
        return np.nan

if 'bathrooms_text' in candidate_numeric:
    df['bathrooms_parsed'] = df['bathrooms_text'].map(parse_bathrooms_text)
    candidate_numeric.remove('bathrooms_text')
    candidate_numeric.append('bathrooms_parsed')

candidate_categorical = [c for c in [
    'room_type','neighbourhood_cleansed','property_type','bed_type','instant_bookable'
] if c in df.columns]

features = candidate_numeric + candidate_categorical
print('Using features:', features)
print('Target: price_clean')

# Drop rows with missing target or extreme outliers (optional cap)
df_model = df[features + ['price_clean']].dropna(subset=['price_clean']).copy()

# Remove non-positive prices
df_model = df_model[df_model['price_clean'] > 0]

# Optional: cap price at 99th percentile to reduce extreme outliers effect
cap = df_model['price_clean'].quantile(0.99)
df_model.loc[df_model['price_clean'] > cap, 'price_clean'] = cap

X = df_model[features]
y = df_model['price_clean']

num_features = [c for c in features if c in candidate_numeric]
cat_features = [c for c in features if c in candidate_categorical]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

X_train.shape, X_test.shape

## 6) Model Training Pipelines

We build pipelines for:
- **Linear Regression**
- **SVR (RBF kernel)** — benefits from scaling
- **Decision Tree Regressor**

In [ ]:
pipe_lin = Pipeline(steps=[
    ('prep', preprocessor),
    ('reg', LinearRegression())
])

pipe_svr = Pipeline(steps=[
    ('prep', preprocessor),
    ('reg', SVR(kernel='rbf'))
])

pipe_dt = Pipeline(steps=[
    ('prep', preprocessor),
    ('reg', DecisionTreeRegressor(random_state=RANDOM_STATE))
])

# Hyperparameter grids
param_grid_lin = {
    # LinearRegression has no major hyperparams; keep empty or add normalize=False (deprecated).
}

param_grid_svr = {
    'reg__C': [1.0, 5.0, 10.0],
    'reg__epsilon': [0.1, 1.0, 5.0],
    'reg__gamma': ['scale', 0.01, 0.001]
}

param_grid_dt = {
    'reg__max_depth': [None, 5, 10, 20, 30],
    'reg__min_samples_split': [2, 5, 10],
    'reg__min_samples_leaf': [1, 2, 5]
}

print('Pipelines and param grids ready.')

## 7) Hyperparameter Tuning (GridSearchCV)

We'll use 3-fold CV for speed. Increase `cv` for more robust tuning.

In [7]:
def run_grid(model, params, name):
    if len(params) == 0:
        # Fit directly if no hyperparameters
        model.fit(X_train, y_train)
        return model, {}, np.nan
    grid = GridSearchCV(model, params, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    print(f'Best {name} params:', grid.best_params_)
    print(f'Best {name} CV RMSE: {-grid.best_score_:.4f}')
    return grid.best_estimator_, grid.best_params_, -grid.best_score_

best_lin, best_lin_params, best_lin_cv = run_grid(pipe_lin, param_grid_lin, 'LinearRegression')
best_svr, best_svr_params, best_svr_cv = run_grid(pipe_svr, param_grid_svr, 'SVR')
best_dt,  best_dt_params,  best_dt_cv  = run_grid(pipe_dt,  param_grid_dt,  'DecisionTree')

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best SVR params: {'reg__C': 10.0, 'reg__epsilon': 5.0, 'reg__gamma': 0.01}
Best SVR CV RMSE: 53.4881
Fitting 3 folds for each of 45 candidates, totalling 135 fits
Best DecisionTree params: {'reg__max_depth': 5, 'reg__min_samples_leaf': 5, 'reg__min_samples_split': 2}
Best DecisionTree CV RMSE: 52.4407


## 8) Training Evaluation

We compute MAE, MSE, RMSE on the **training set** for each tuned model.

In [8]:
def regress_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mae, mse, rmse

def evaluate_train(name, model):
    yhat = model.predict(X_train)
    mae, mse, rmse = regress_metrics(y_train, yhat)
    print(f'{name}: MAE={mae:.3f} | MSE={mse:.3f} | RMSE={rmse:.3f}')
    return {'model': name, 'train_mae': mae, 'train_mse': mse, 'train_rmse': rmse}

train_rows = []
train_rows.append(evaluate_train('Linear Regression (best)', best_lin))
train_rows.append(evaluate_train('SVR (best)', best_svr))
train_rows.append(evaluate_train('Decision Tree (best)', best_dt))

train_df = pd.DataFrame(train_rows)
display(train_df)

Linear Regression (best): MAE=32.590 | MSE=2240.834 | RMSE=47.337
SVR (best): MAE=32.921 | MSE=2690.142 | RMSE=51.867
Decision Tree (best): MAE=33.216 | MSE=2360.201 | RMSE=48.582


,model,train_mae,train_mse,train_rmse
0,Linear Regression (best),32.590151,2240.834374,47.337452
1,SVR (best),32.921299,2690.141524,51.866574
2,Decision Tree (best),33.215584,2360.200929,48.581899


## 9) Testing Evaluation + Bootstrapping 95% CIs

We evaluate on the held-out test set and use bootstrapping to compute the mean MAE/MSE/RMSE and their 95% confidence intervals.

In [10]:
def bootstrap_errors(y_true, y_pred, n_boot=500, random_state=RANDOM_STATE):
    rng = np.random.RandomState(random_state)
    n = len(y_true)
    maes, mses, rmses = [], [], []
    base = {}
    # Base (full) errors
    base['mae'], base['mse'], base['rmse'] = regress_metrics(y_true, y_pred)
    for _ in range(n_boot):
        idx = rng.randint(0, n, size=n)
        yt = y_true[idx]
        yp = y_pred[idx]
        m = mean_absolute_error(yt, yp)
        s = mean_squared_error(yt, yp)
        r = np.sqrt(s)
        maes.append(m); mses.append(s); rmses.append(r)
    def ci(a):
        low, high = np.percentile(a, [2.5, 97.5])
        return float(np.mean(a)), float(low), float(high)
    mae_m, mae_l, mae_h = ci(maes)
    mse_m, mse_l, mse_h = ci(mses)
    rmse_m, rmse_l, rmse_h = ci(rmses)
    return base, {
        'mae_mean': mae_m, 'mae_ci_low': mae_l, 'mae_ci_high': mae_h,
        'mse_mean': mse_m, 'mse_ci_low': mse_l, 'mse_ci_high': mse_h,
        'rmse_mean': rmse_m, 'rmse_ci_low': rmse_l, 'rmse_ci_high': rmse_h
    }

test_rows = []
for name, model in [
    ('Linear Regression (best)', best_lin),
    ('SVR (best)', best_svr),
    ('Decision Tree (best)', best_dt)
]:
    yhat_test = model.predict(X_test)
    base, ci = bootstrap_errors(y_test.values, yhat_test, n_boot=500)
    row = {'model': name}
    row.update({f'test_{k}': v for k, v in base.items()})
    row.update(ci)
    test_rows.append(row)

test_df = pd.DataFrame(test_rows)
display(test_df)

,model,test_mae,test_mse,test_rmse,mae_mean,mae_ci_low,mae_ci_high,mse_mean,mse_ci_low,mse_ci_high,rmse_mean,rmse_ci_low,rmse_ci_high
0,Linear Regression (best),32.750380,2472.989168,49.729158,32.767991,29.950645,35.329123,2479.486653,1931.011675,3088.137116,49.716513,43.943252,55.570930
1,SVR (best),32.618741,2774.286854,52.671499,32.634184,29.649240,35.420631,2778.759798,2093.601847,3507.249848,52.612980,45.755889,59.222037
2,Decision Tree (best),34.960839,2828.525093,53.183880,34.977034,32.237524,37.779738,2837.474061,2195.197643,3525.638462,53.177115,46.852871,59.377042


## 10) Model Comparison & Discussion

We merge training and testing metrics and flag potential overfitting (large train–test gaps).

In [11]:
merged = pd.merge(train_df, test_df, on='model', how='inner')
display(merged)

def gap_note(train_rmse, test_rmse, thresh=10.0):
    # Rule of thumb: if test RMSE exceeds train RMSE by more than a threshold, flag overfitting.
    gap = test_rmse - train_rmse
    if gap > thresh:
        return 'Potential overfitting (test RMSE >> train RMSE).'
    elif gap < -1.0:
        return 'Test < Train — possible underfitting metrics artifact or strong regularization.'
    else:
        return 'No strong over/underfitting signal from RMSE gap.'

for _, r in merged.iterrows():
    print(f"{r['model']}: Train RMSE={r['train_rmse']:.2f} | Test RMSE={r['test_rmse']:.2f} => {gap_note(r['train_rmse'], r['test_rmse'])}")

,model,train_mae,train_mse,train_rmse,test_mae,test_mse,test_rmse,mae_mean,mae_ci_low,mae_ci_high,mse_mean,mse_ci_low,mse_ci_high,rmse_mean,rmse_ci_low,rmse_ci_high
0,Linear Regression (best),32.590151,2240.834374,47.337452,32.750380,2472.989168,49.729158,32.767991,29.950645,35.329123,2479.486653,1931.011675,3088.137116,49.716513,43.943252,55.570930
1,SVR (best),32.921299,2690.141524,51.866574,32.618741,2774.286854,52.671499,32.634184,29.649240,35.420631,2778.759798,2093.601847,3507.249848,52.612980,45.755889,59.222037
2,Decision Tree (best),33.215584,2360.200929,48.581899,34.960839,2828.525093,53.183880,34.977034,32.237524,37.779738,2837.474061,2195.197643,3525.638462,53.177115,46.852871,59.377042


Linear Regression (best): Train RMSE=47.34 | Test RMSE=49.73 => No strong over/underfitting signal from RMSE gap.
SVR (best): Train RMSE=51.87 | Test RMSE=52.67 => No strong over/underfitting signal from RMSE gap.
Decision Tree (best): Train RMSE=48.58 | Test RMSE=53.18 => No strong over/underfitting signal from RMSE gap.
